In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import gc

In [ ]:
!wc -l /storage/data/simple_features.csv

In [ ]:
total_items = 1590000
validation_split = 0.3
data_chunksize = 100_000
text_chunksize = 10_000
images_chunksize = 10_000

In [ ]:
def fill_empty_images(row, empty_image):
  row = row.to_dict()
  if row['_merge'] == 'left_only':
    row.update(empty_image)
  return pd.Series(row)

In [ ]:
data_chunks = pd.read_csv('/storage/data/simple_features.csv', chunksize=data_chunksize)

In [ ]:
# as data was already presorted by date so we can just mark first rows for train and last rows for validation
for i, data_chunk in enumerate(data_chunks):
  test_chunk = total_items * (1 - validation_split) < i * data_chunksize
  indexes = data_chunk['item_id'].tolist()

  text_chunks = pd.read_csv('/storage/data/text_train.csv', chunksize=text_chunksize)
  dataframes = []
  for j, text_chunk in enumerate(text_chunks):
    dataframes.append(text_chunk[text_chunk['item_id'].isin(indexes)])
  data_chunk = data_chunk.merge(pd.concat(dataframes, axis=0), on='item_id', how='left')
  dataframes = None
  gc.collect()

  images_chunks = pd.read_csv('/storage/data/images_resnet.csv', names=['image_id'] + ['image_{}'.format(i) for i in range(2048)],
                              chunksize=images_chunksize)
  image_indexes = data_chunk['image'].tolist()
  dataframes = []
  for j, image_chunk in enumerate(images_chunks):
    dataframes.append(image_chunk[image_chunk['image_id'].isin(image_indexes)])
    empty_image_df = image_chunk[image_chunk['image_id'] == 'empty']
    if len(empty_image_df) > 0:
      empty_image = empty_image_df.iloc[0].to_dict()
  data_chunk = data_chunk.merge(pd.concat(dataframes, axis=0), 
                                left_on='image', right_on='image_id', how='left', indicator='_merge')
  data_chunk = data_chunk.apply(lambda row: fill_empty_images(row, empty_image), axis=1)
  data_chunk = data_chunk.drop('_merge', axis=1)
  dataframes = None

  n_file = i == 0 or (test_chunk and total_items * (1 - validation_split) >= (i-1) * data_chunksize)
  data_chunk.to_csv('/storage/data/dataset_{}'.format('valid' if test_chunk else 'train'), 
                    header = n_file, index=False, mode = 'w' if n_file else 'a')
  data_chunk = None
  gc.collect()
